In [1]:
%load_ext autoreload
%autoreload 2

# Preliminaries

In [1]:
%pip install markdownify python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Load the Astra Documentation into Knowledge Store

First, we'll crawl the DataStax documentation. LangChain includes a `SiteMapLoader` but it loads all of the pages into memory simultaneously, which makes it impossible to index larger sites from small environments (such as CoLab). So, we'll scrape the sitemap ourselves and iterate over the URLs, allowing us to process documents in batches and flush them to Astra DB. 

## Scrape the URLs from the Site Maps
First, we use Beautiful Soup to parse the XML content of each sitemap and get the list of URLs.
We also add a few extra URLs for external sites that are also useful to include in the index.

In [9]:
# Use sitemaps to crawl the content
SITEMAPS = [
    "https://docs.datastax.com/en/sitemap-astra-db-vector.xml",
    "https://docs.datastax.com/en/sitemap-cql.xml",
    "https://docs.datastax.com/en/sitemap-dev-app-drivers.xml",
    "https://docs.datastax.com/en/sitemap-glossary.xml",
    "https://docs.datastax.com/en/sitemap-astra-db-serverless.xml"
]

# Additional URLs to crawl for content.
EXTRA_URLS = [
    "https://github.com/jbellis/jvector"
]

SITE_PREFIX = "astra"

from bs4 import BeautifulSoup
import requests

def load_pages(sitemap_url):
    r = requests.get(sitemap_url,
                     headers={
                         # Astra docs only return a sitemap with a user agent set.
                         "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0",
                     })
    xml = r.text

    soup = BeautifulSoup(xml, features="xml")
    url_tags = soup.find_all("url")
    for url in url_tags:
        yield(url.find("loc").text)


# For maintenance purposes, we could check only the new articles since a given time.
URLS = [
    url
    for sitemap_url in SITEMAPS
    for url in load_pages(sitemap_url)
] + EXTRA_URLS
len(URLS)

1368

## Load the content from each URL
Next, we create the code to load each page. This performs the following steps:

1. Parses the HTML with BeautifulSoup
2. Locates the "content" of the HTML using an appropriate selector based on the URL
3. Find the link (`<a href="...">`) tags in the content and collect the absolute URLs (for creating edges).

Adding the URLs of these references to the metadata allows the knowledge store to create edges between the document.

In [10]:
from langchain_community.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from typing import AsyncIterator, Iterable, Set
from markdownify import MarkdownConverter
from urllib.parse import urlparse, urljoin, urldefrag

def parse_url(link,
              page_url,
              drop_fragment: bool = True):
  href = link.get('href')
  if href is None:
    return None
  url = urlparse(href)
  if url.scheme not in ["http", "https", ""]:
    return None

  # Join the HREF with the page_url to convert relative paths to absolute.
  url = urljoin(page_url, href)

  # Fragments would be useful if we chunked a page based on section.
  # Then, each chunk would have a different URL based on the fragment.
  # Since we aren't doing that yet, they just "break" links. So, drop
  # the fragment.
  if drop_fragment:
    return urldefrag(url).url
  else:
     return url

def parse_hrefs(soup: BeautifulSoup, url: str) -> Set[str]:
  links = soup.find_all('a')
  links = {parse_url(link, page_url=url) for link in links}

  # Remove entries for any 'a' tag that failed to parse (didn't have href,
  # or invalid domain, etc.)
  links.discard(None)

  # Remove self links.
  links.discard(url)

  return links

def locate_content(soup: BeautifulSoup, url: str) -> BeautifulSoup:
    content = None
    if url.startswith("https://docs.datastax.com/en/"):
        content = soup.select_one("article.doc")
    if url.startswith("https://github.com"):
        content = soup.select_one("article.entry-content")
    assert content is not None, f"Unable to locate content for {url}"
    return content

markdown_converter = MarkdownConverter(autolinks_false="false", heading_style="ATX")

def process_document(html: Document) -> Document:
    url = html.metadata["source"]
    soup = BeautifulSoup(html.page_content, "html.parser")
    content = locate_content(soup, url)

    content_md = markdown_converter.convert_soup(content)
    hrefs = parse_hrefs(content, url)
    return Document(
        page_content = content_md,
        metadata = {
            # Assign the unique ID for the `Document` in the graph.
            "content_id": url,
            # This document references all documents with matching urls.
            "hrefs": hrefs,
            # These are the URLs the document is "defined" at.
            "urls": [url]
        }
    )


async def load_pages(urls: Iterable[str]) -> AsyncIterator[Document]:
    loader = AsyncHtmlLoader(
        urls,
        requests_per_second=4,
        # Astra docs require a user agent
        header_template = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0"
        }
    )
    async for html in loader.alazy_load():
        yield process_document(html)

## Initialize Environment
Before we initialize the Knowledge Store and write the documents we need to set some environment variables.
In colab, this will prompt you for input. When running locally, this will load from `.env`.

In [2]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    # (Option 1) - Set the environment variables from getpass.
    print("In colab. Using getpass/input for environment variables.")
    import getpass
    import os

    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")
    os.environ["ASTRA_DB_DATABASE_ID"] = input("Enter Astra DB Database ID: ")
    os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass("Enter Astra DB Application Token: ")

    keyspace = input("Enter Astra DB Keyspace (Empty for default): ")
    if keyspace:
        os.environ["ASTRA_DB_KEYSPACE"] = keyspace
    else:
        os.environ.pop("ASTRA_DB_KEYSPACE", None)
else:
    print("Not in colab. Loading '.env' (see 'env.template' for example)")
    import dotenv
    dotenv.load_dotenv()

Not in colab. Loading '.env' (see 'env.template' for example)


## Initialize Cassio and Knowledge Store
With the environment variables set we initialize the Cassio library for talking to Cassandra / Astra DB.
We also create the `KnowledgeStore`.

In [4]:
answer = input("Drop Tables? [(Y)es/(N)o]")
if answer.lower() in ["y","yes"]:
    import cassio
    cassio.init(auto=True)
    from cassio.config import check_resolve_session, check_resolve_keyspace
    session = check_resolve_session()
    keyspace = check_resolve_keyspace()
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_nodes")
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_edges")
else:
     # Handle no / "wrong" input

In [11]:
import cassio
from langchain_openai import OpenAIEmbeddings
from ragstack_knowledge_store import CassandraKnowledgeStore
from ragstack_knowledge_store.directed_edge_extractor import DirectedEdgeExtractor

cassio.init(auto=True)
embeddings = OpenAIEmbeddings()
SITE_PREFIX="astra_docs"
knowledge_store = CassandraKnowledgeStore(
    embeddings,
    edge_extractors = [
        DirectedEdgeExtractor.for_hrefs_to_urls(),
    ],
    node_table=f"{SITE_PREFIX}_nodes",
    edge_table=f"{SITE_PREFIX}_edges")

## Load the Documents
Finally, we fetch pages and write them to the knowledge store in batches of 50.

In [12]:
not_found = 0
found = 0

docs = []
async for doc in load_pages(URLS):
    if doc.page_content.startswith("\n# Page Not Found"):
        not_found += 1
        continue

    docs.append(doc)
    found += 1

    if len(docs) >= 50:
        knowledge_store.add_documents(docs)
        docs.clear()

if docs:
    knowledge_store.add_documents(docs)
print(f"{not_found} (of {not_found + found}) URLs were not found")

Fetching pages: 100%|##########| 1368/1368 [04:24<00:00,  5.18it/s]


96 (of 1368) URLs were not found


# Create and execute the RAG Chains

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o")

template = """You are a helpful technical support bot. You should provide complete answers explaining the options the user has available to address their problem. Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    formatted = "\n\n".join(f"From {doc.metadata['content_id']}: {doc.page_content}" for doc in docs)
    return formatted

We'll use the following question. This is an interesting question because the ideal answer should be concise and in-depth, based on how the vector indexing is actually implemented.

In [6]:
QUESTION="What vector indexing algorithms does Astra use?"

In [24]:
from IPython.display import display, Markdown

# Helper method to render markdown in responses to a chain.
def run_and_render(chain, question):
    result = chain.invoke(question)
    display(Markdown(result))

## Vector-Only Retrieval

In [25]:
# Depth 0 doesn't traverses edges and is equivalent to vector similarity only.
vector_retriever = knowledge_store.as_retriever(search_kwargs={"depth": 0})

vector_rag_chain = (
    {"context": vector_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
run_and_render(vector_rag_chain, QUESTION)

Astra DB uses multiple indexing techniques to speed up searches in its vector databases. The key vector indexing algorithms used are:

1. **JVector**: Astra DB Serverless (Vector) databases utilize the JVector vector search engine to construct a graph index. This engine enables efficient search operations by adding new documents to the graph immediately. JVector also supports vector compression through quantization to save space and improve performance. For more details, you can visit the [JVector GitHub page](https://github.com/jbellis/jvector).

2. **Storage-Attached Index (SAI)**: This indexing technique is used to efficiently find rows that satisfy query predicates. Astra DB provides numeric, text, and vector-based indexes to support various types of searches. SAI helps in loading a superset of all possible results from storage based on the provided predicates, evaluating the search criteria, and sorting the results by vector similarity. The top results are then returned to the user. For more information, see the [Storage-Attached Indexing (SAI) Overview](https://docs.datastax.com/en/cql/astra/developing/indexing/sai/sai-overview.html).

These indexing techniques help in optimizing the performance and efficiency of vector searches in Astra DB.

## Graph Traversal Retrieval

In [27]:
# Depth 1 does vector similarity and then traverses 1 level of edges.
graph_retriever = knowledge_store.as_retriever(search_kwargs={"depth": 1})

graph_rag_chain = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
run_and_render(graph_rag_chain, QUESTION)

Astra DB Serverless uses multiple indexing techniques to optimize vector searches. Here are the key indexing methods employed:

1. **JVector**: Astra uses the [JVector vector search engine](https://github.com/jbellis/jvector) to construct a graph index. JVector builds a graph-based index that enables efficient and scalable vector searches. When new documents are added, JVector immediately integrates them into the graph, allowing for prompt search capabilities. JVector also supports vector compression with techniques like quantization to save space and improve performance.

2. **Storage-Attached Index (SAI)**: SAI is another indexing technique used by Astra DB to efficiently search rows that satisfy query predicates. Astra DB provides numeric-, text-, and vector-based indexes to support various search operations. SAI can be customized according to specific requirements, such as a particular similarity function or text transformation. When a search operation is executed, SAI loads a superset of potential results from storage based on the provided predicates and then evaluates and sorts these results by vector similarity. The top `limit` results are returned to the user.

These indexing techniques help Astra DB Serverless perform efficient and scalable vector searches, ensuring that queries return relevant results quickly.

## MMR Graph Traversal

In [29]:
mmr_graph_retriever = knowledge_store.as_retriever(
    search_type = "mmr_traversal",
    search_kwargs = {
        "k": 4,
        "fetch_k": 10,
        "depth": 2,
        # "score_threshold": 0.2,
    },
)

mmr_graph_rag_chain = (
    {"context": mmr_graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
run_and_render(mmr_graph_rag_chain, QUESTION)

Astra DB Serverless uses multiple indexing techniques to speed up vector searches, with a focus on efficient and scalable solutions. Specifically, Astra employs the following vector indexing algorithms:

1. **JVector**:
   - **Graph Index**: Astra DB uses the [JVector vector search engine](https://github.com/jbellis/jvector) to construct a graph index. JVector is a graph-based index that builds on the DiskANN design with composable extensions.
   - **Graph-based Indexes**: These indexes are simpler to implement and typically faster. They can be constructed and updated incrementally, making them more suitable for general-purpose indexing compared to partition-based approaches.
   - **On-Disk Adjacency List**: The graph is represented by an on-disk adjacency list per node, with additional data stored inline to support two-pass searches.
   - **Two-Pass Searches**: The first pass uses lossily compressed representations of the vectors kept in memory, and the second pass uses a more accurate representation read from disk.

2. **Product Quantization (PQ)**:
   - **Compression**: JVector can compress vectors using product quantization to save space and improve performance.
   - **Anisotropic Weighting**: Optionally used with product quantization to improve recall.

3. **Storage-Attached Index (SAI)**:
   - **Efficient Searching**: SAI is an indexing technique used to efficiently find rows that satisfy query predicates. It supports numeric-, text-, and vector-based indexes.
   - **Customization**: You can customize indexes based on specific requirements, such as using a specific similarity function or text transformation.

These indexing techniques enable Astra DB Serverless to provide efficient and scalable vector search capabilities, suitable for various high-dimensional data scenarios often encountered in machine learning and AI applications.

## Check Retrieval Results

In [31]:
# Set the question and see what documents each technique retrieves.
QUESTION="What vector indexing algorithms does Astra use?"
for i, doc in enumerate(vector_retriever.invoke(QUESTION)):
  print(f"Vector [{i}]:    {doc.metadata['content_id']}")

for i, doc in enumerate(graph_retriever.invoke(QUESTION)):
  print(f"Graph [{i}]:     {doc.metadata['content_id']}")

for i, doc in enumerate(mmr_graph_retriever.invoke(QUESTION)):
  print(f"MMR Graph [{i}]: {doc.metadata['content_id']}")

Vector [0]:    https://docs.datastax.com/en/astra-db-serverless/get-started/concepts.html
Vector [1]:    https://docs.datastax.com/en/cql/astra/getting-started/vector-search-quickstart.html
Vector [2]:    https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html
Vector [3]:    https://docs.datastax.com/en/astra-db-serverless/get-started/astra-db-introduction.html


Graph [0]:     https://docs.datastax.com/en/astra-db-serverless/get-started/concepts.html
Graph [1]:     https://docs.datastax.com/en/cql/astra/getting-started/vector-search-quickstart.html
Graph [2]:     https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html
Graph [3]:     https://docs.datastax.com/en/astra-db-serverless/get-started/astra-db-introduction.html
Graph [4]:     https://docs.datastax.com/en/astra-db-serverless/databases/load-data.html
Graph [5]:     https://docs.datastax.com/en/astra-db-serverless/databases/manage-collections.html
Graph [6]:     https://docs.datastax.com/en/astra-db-serverless/integrations/embedding-providers/azure-openai.html
Graph [7]:     https://docs.datastax.com/en/astra-db-serverless/integrations/embedding-providers/nvidia-nemo.html
Graph [8]:     https://docs.datastax.com/en/astra-db-serverless/integrations/embedding-providers/openai.html
Graph [9]:     https://docs.datastax.com/en/astra-db-serverless/databases/database

# Conclusion
With vector only we retrieved chunks from the Astra documentation explaining that it used JVector.
Since it didn't follow the link to [JVector on GitHub](https://github.com/jbellis/jvector) it didn't actually answer the question.

The graph retrieval started with the same set of chunks, but it followed the edge to the documents we loaded from GitHub.
This allowed the LLM to read in more depth how JVector is implemented, which allowed it to answer the question more clearly and with more detail.